Mixed models with variance components
------------------------------------------

This notebook demonstrates fitting mixed linear models with variance components in Statsmodels.

In [0]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
import patsy

### Specifying models

Statsmodels MixedLM is group-based, meaning that the data are partitioned into specified, disjoint groups, and all random effects are independently realized across the groups.  Crossed random effects that are shared over the groups are not supported at present.

The interface to `MixedLM` involves three types of effects.  

* Fixed effects are coefficients that multiply the columns of a design matrix (`exog_fe`), just like in a traditional linear model.

* Random effects are random terms that are arbitrarily correlated with each other.  There must be the same number of random terms for every group.  For example, if there are two random effects `r1` and `r2`, then the random effects design matrix `exog_re` must have two columns `v1` and `v2`.  The linear combination `r1*v1 + r2*v2` is an additive term contributing to `endog`.  Note that `r1` and `r2` are random variables with mean zero that are realized independently for each group.  There are three unknown parameters describing this part of the model: variance parameters for `r1` and `r2`, and a parameter for the covariance between `r1` and `r2`.

* Variance components are random terms with mean zero that are always uncorrelated with each other.  Each variance component has a single parameter associated with it defining its variance.  There may be multiple independent realizations of each variance component within a group, and different groups can have differing numbers of realizations of each variance component.  

One use case for variance components is nested random intercepts.  Suppose the top-level group is the school that a student attends, and there are dfferent classrooms within each school.  The random intercepts for the classrooms should be independent of each other, but all have the same variance.  The number of classrooms per school may vary. Extending this idea, a variance component could represent a random coefficient for an observed variable (like the score on a previous test).  If these slopes vary by classroom then they can be modeled as variance components. 

The standard array-based interface for specifying variance components is somewhat complex, it is somewhat easier to use formulas.  We will first describe the array-based interface first and then describe the formula interface below.  

Suppose we have two variance components, one a random intercept for classrooms, and one a random coefficient for the previous year's test score, which we want to allow to vary either by school or by classroom.  We pass to `MixedLM` a dictionary `vc` containing the design information for the variance components.  The keys for `vc` are names for the variance components, here 'classroom' and 'prior_score'.  

The classroom effect design information for group 0 is located in ``vc['classroom'][0]``.  It could be an array such as:

```
1 0 0
1 0 0
1 0 0
0 1 0
0 1 0
0 1 0
0 0 1
0 0 1
0 0 1
```

Note that this array has three columns, indicating that there are three classrooms in school 0.  The columns are indicator vectors reflecting which student attends each school.  Note that since different schools could have different numbers of classrooms, these design matrices may have differing numbers of columns.  Regardless of the number of columns, all random coefficients multiplying the columns of these matrices will have the same variance parameter.

The second variance component is for the prior year's test score.  If we want this variable to have a random coefficient that varies by school but not by classroom, it could be included in `exog_re`, rather than treating it as a variance component (this would result in a random slope for the prior test score that is correlated with other random terms in the model).  Alternatively, the prior test score could be modeled using a variance component.  The resulting random terms would be uncorrelated with any other random terms in the model.  If we set ``vc['priortest'][0]`` equal to a column vector of prior test score, like

```
76
89
56
77
94
85
83
78
82
```

then the random coefficient for the prior test score is the same for everyone in a given school.  If instead we set ``vc['priortest'][0]`` to

```
76  0  0
89  0  0
56  0  0
0  77  0
0  94  0
0  85  0
0   0 83
0   0 78
0   0 82
```

then each classroom has its own slope for the prior test scores.  These slopes are independent of each other but all have have the same variance.

The following function simulates data following the example discussed above.

In [0]:
def gen_data(n):
    
    # The fixed effects design matrix
    exog = np.random.normal(size=(n, 3))

    # A random intercept and two covariates with random slopes.
    exog_re = np.random.normal(size=(n, 2))
    exog_re[:, 0] = 1
    
    # Group labels for two levels of nested subgroups.
    exog_vc = np.empty((n, 2))
    exog_vc[:, 0] = np.kron(np.arange(n / 4), np.ones(4)) # Classroom membership
    exog_vc[:, 0] = exog_vc[:, 0] % 4                     # Restart labels in each school
    exog_vc[:, 1] = np.random.normal(size=n)              # Prior test score
    
    # All groups have 16 observations
    groups = np.kron(np.arange(n / 16), np.ones(16))
    
    # Build up the error term
    errors = 0.
    
    # Add random effects to the error term
    exog_coeffs = np.random.normal(size=(n / 16, 2))
    exog_coeffs = np.kron(exog_coeffs, np.ones((16, 1)))
    errors += (exog_re * exog_coeffs).sum(1)
    
    # True variance component parameters
    vcomp = [2, 5]
    
    # Add variance components to the error term
    for j in 0,1:
        for k in range(4): # 4 classrooms per school
            x = (exog_vc[:, 0] == k)
            if j == 1:
                x = x * exog_vc[:, 1] # prior test score for one classroom
            rt = np.random.normal(size=n/16) * np.sqrt(vcomp[j])
            rt = np.kron(rt, np.ones(16))
            errors += rt * x 
    
    # Add iid 'residual' errors to the error term
    errors += np.random.normal(size=n)
    
    endog = np.dot(exog, np.r_[0, 1, -2])
    endog += errors
    
    return endog, groups, exog, exog_re, exog_vc

Next we generate data, and take a look at what we have in `exog_vc`:

In [0]:
endog, groups, exog, exog_re, exog_vc = gen_data(1600)

print(exog_vc[0:16, :])

The `exog_vc` data needs further processing.  We use Patsy to help with this, as shown below.  Note that when constructing a subgroup random intercept or slope, you should not include an intercept in the model (by using "0 +" in the formula).

In [0]:
df = pd.DataFrame(exog_vc, columns=("classroom", "priortest"))

vca = patsy.dmatrix("0 + C(classroom)", df, return_type='dataframe')
vcb = patsy.dmatrix("0 + C(classroom):priortest", df, return_type='dataframe')

vc = {"classroom": {}, "priortest": {}}
for k in range(100):
    ii = np.flatnonzero(groups == k)
    vc["classroom"][k] = np.asarray(vca.iloc[ii, :])
    vc["priortest"][k] = np.asarray(vcb.iloc[ii, :])

print(vc["classroom"][0])
print(vc["priortest"][0])

Now we can fit the model.

In [0]:
ex = sm.add_constant(exog)
model1 = sm.MixedLM(endog, ex, groups, exog_re=exog_re, exog_vc=vc)
result1 = model1.fit()
print(result1.summary())

Next we fit the same model using formulas.  Note again that subgroup-level intercepts and slopes should not have overall intercepts, so use "0 +" in the formula.

In [0]:
df = pd.DataFrame(index=range(len(endog)))
df["y"] = endog
df["x1"] = exog[:, 0]
df["x2"] = exog[:, 1]
df["x3"] = exog[:, 2]
df["z1"] = exog_re[:, 0]
df["z2"] = exog_re[:, 1]
df["classroom"] = exog_vc[:, 0]
df["prior_test"] = exog_vc[:, 1]
df["groups"] = groups

vcf = {"classroom" : "0 + C(classroom)", "prior_test" : "0 + C(classroom):prior_test"}

model2 = sm.MixedLM.from_formula("y ~ x1 + x2 + x3", groups="groups", re_formula="0 + z1 + z2", vc_formula=vcf, data=df)
result2 = model2.fit()
print(result2.summary())